# Prepare Dataset

In [294]:
import numpy as np
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeClassifierCV
from xgboost import XGBClassifier, XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import Input, Embedding, LSTM, Dense, SimpleRNN, Attention, Concatenate
from keras.models import Model, Sequential, load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.layers.wrappers import Bidirectional
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import pandas as pd
from math import log
import numpy as np
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
dir = '/content/drive/My Drive/IST557/HW4/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [202]:
data_train = np.load(dir+'train.npz') 
x_train = data_train['x'] #feature matrix
y_train = data_train['y'] #label matrix
location_train = data_train['locations'] #location matrix
times_train = data_train['times'] #time matrix

data_val = np.load(dir+'val.npz') 
x_val = data_val['x'] #feature matrix
y_val = data_val['y'] #label matrix
location_val = data_val['locations'] #location matrix
times_val = data_val['times'] #time matrix

data_test = np.load(dir+'test.npz') 
x_test = data_test['x'] #feature matrix
location_test = data_test['locations'] #location matrix
times_test = data_test['times'] #time matrix

In [203]:
x_train.shape, y_train.shape, location_train.shape, times_train.shape

((72000, 8, 49), (72000, 1), (72000, 2), (72000,))

In [204]:
len(x_train), len(x_val), len(x_test)

(72000, 18000, 1600)

# Q1) simple baseline
Implement a simple baseline where the historical average for each region is predicted against the validation set.

In [157]:
y_train_mean = x_train[:, :, 24].mean(1)

region_dict = {}
for i, value in enumerate(y_train_mean):
  region = i%100
  try:
    region_dict[region].append(value)
  except Exception:
    region_dict[region] = [value]

In [158]:
base = [np.mean(region_values) for region_values in region_dict.values()]

In [159]:
baseline_pred = []
for i, value in enumerate(x_val):
  region = i%100
  baseline_pred.append(base[region])

In [160]:
mean_squared_error(y_val, baseline_pred, squared=False)

0.05450973121446298

# Q2) linear regression
Extract features from the temporal data and use linear regression to predict demand on the validation set.

In [71]:
X_train = x_train.reshape(len(x_train), 8*49)
X_val = x_val.reshape(len(x_val), 8*49)

In [23]:
X_train.shape, y_train.shape, X_val.shape

((72000, 392), (72000, 1), (18000, 392))

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_val_pred_lr = lr.predict(X_val)

In [25]:
mean_squared_error(y_val, y_val_pred_lr, squared=False)

0.02646154744403434

# Q3) XGBoost
Extract features from the temporal data and use XGBOOST to predict demand on the validation set.

In [72]:
xgb = XGBRegressor(n_estimators=1000)
xgb.fit(X_train, y_train)
y_val_pred_xgb = xgb.predict(X_val)

[22:00:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [77]:
mean_squared_error(y_val, y_val_pred_xgb, squared=False)

0.023051841562040234

# Q4) RNN
Implement a basic Recurrent Neural Network (RNN) to predict demand on the validation set.

In [228]:
x_train.shape

(72000, 8, 49)

In [230]:
rnn = Sequential()
rnn.add(SimpleRNN(32, activation='tanh', input_shape=(x_train.shape[1], x_train.shape[2])))
rnn.add(Dense(16))
rnn.add(Dense(1))
rnn.compile(optimizer='adam', loss='mean_squared_error')
rnn.summary()

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 32)                2624      
_________________________________________________________________
dense_131 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_132 (Dense)            (None, 1)                 17        
Total params: 3,169
Trainable params: 3,169
Non-trainable params: 0
_________________________________________________________________


In [40]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3)
rnn.fit(x_train, y_train, batch_size=100, epochs=500, verbose=1, callbacks=[early_stopping], validation_data=(x_val, y_val))

Epoch 1/500
720/720 [==============================] - 2s 3ms/step - loss: 0.0029 - val_loss: 9.8495e-04
Epoch 2/500
720/720 [==============================] - 2s 3ms/step - loss: 8.0642e-04 - val_loss: 7.9776e-04
Epoch 3/500
720/720 [==============================] - 2s 3ms/step - loss: 7.3242e-04 - val_loss: 7.7394e-04
Epoch 4/500
720/720 [==============================] - 2s 3ms/step - loss: 7.1399e-04 - val_loss: 7.3798e-04
Epoch 5/500
720/720 [==============================] - 2s 3ms/step - loss: 7.0560e-04 - val_loss: 7.2421e-04
Epoch 6/500
720/720 [==============================] - 2s 3ms/step - loss: 7.0192e-04 - val_loss: 7.1737e-04
Epoch 7/500
720/720 [==============================] - 2s 3ms/step - loss: 6.9802e-04 - val_loss: 7.2867e-04
Epoch 8/500
720/720 [==============================] - 2s 3ms/step - loss: 7.0174e-04 - val_loss: 7.7085e-04
Epoch 9/500
720/720 [==============================] - 2s 3ms/step - loss: 6.9370e-04 - val_loss: 7.3245e-04
Epoch 10/500
720/720 [=

In [41]:
y_val_pred_rnn = rnn.predict(x_val)
print(mean_squared_error(y_val, y_val_pred_rnn, squared=False))

0.02393130878101878


# Q5) LSTM
Implement a basic Long-short Term Memory (LSTM) network to predict demand on the validation set.

In [14]:
lstm = Sequential()
lstm.add(LSTM(32, activation='tanh', input_shape=(x_train.shape[1], x_train.shape[2])))
lstm.add(Dense(16))
lstm.add(Dense(1))
lstm.compile(optimizer='adam', loss='mean_squared_error')
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                10496     
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 11,041
Trainable params: 11,041
Non-trainable params: 0
_________________________________________________________________


In [44]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3)
lstm.fit(x_train, y_train, batch_size=100, epochs=500, verbose=1, callbacks=[early_stopping], validation_data=(x_val, y_val))

Epoch 1/500
720/720 [==============================] - 4s 6ms/step - loss: 0.0011 - val_loss: 7.1126e-04
Epoch 2/500
720/720 [==============================] - 4s 5ms/step - loss: 6.8031e-04 - val_loss: 7.0235e-04
Epoch 3/500
720/720 [==============================] - 4s 5ms/step - loss: 6.5207e-04 - val_loss: 6.5858e-04
Epoch 4/500
720/720 [==============================] - 4s 5ms/step - loss: 6.2893e-04 - val_loss: 7.3596e-04
Epoch 5/500
720/720 [==============================] - 4s 5ms/step - loss: 6.0927e-04 - val_loss: 6.7863e-04
Epoch 6/500
720/720 [==============================] - 4s 5ms/step - loss: 5.8890e-04 - val_loss: 5.9758e-04
Epoch 7/500
720/720 [==============================] - 4s 5ms/step - loss: 5.7734e-04 - val_loss: 5.9081e-04
Epoch 8/500
720/720 [==============================] - 4s 5ms/step - loss: 5.5467e-04 - val_loss: 5.7484e-04
Epoch 9/500
720/720 [==============================] - 4s 5ms/step - loss: 5.4125e-04 - val_loss: 5.7446e-04
Epoch 10/500
720/720 [=

In [46]:
y_val_pred_lstm = lstm.predict(x_val)
print(mean_squared_error(y_val, y_val_pred_lstm, squared=False))

0.025198831672970764


# Q6) Best Model
Choose the best performing model and predict taxi volume using the test data. Discuss why you chose this model.

## CNN

In [14]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((72000, 8, 49), (72000, 1), (18000, 8, 49), (18000, 1))

In [55]:
x_train_cnn = x_train.reshape(72000, 8, 7, 7).transpose(0,2,3,1)
x_val_cnn = x_val.reshape(18000, 8, 7, 7).transpose(0,2,3,1)

In [182]:
cnn = Sequential()
cnn.add(Conv2D(
          filters = 64, 
          kernel_size = (3, 3), 
          strides = (1, 1),
          padding = 'same', 
          activation = 'relu', 
          input_shape = (x_train_cnn.shape[1], x_train_cnn.shape[2], x_train_cnn.shape[3])
          ))
cnn.add(Conv2D(
          filters = 64, 
          kernel_size = (3, 3), 
          strides = (1, 1),
          activation = 'relu'
          ))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(
          filters = 128,
          kernel_size = (3, 3),
          strides = (1, 1),
          padding = 'same', 
          activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Dropout(0.2))
cnn.add(Flatten())
cnn.add(Dense(256, activation = 'relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64))
cnn.add(Dense(1, activation = 'relu'))
cnn.compile(
      optimizer = 'adam', 
      loss = 'mean_squared_error', 
      metrics = ['mean_squared_error']
      )
cnn.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_88 (Conv2D)           (None, 7, 7, 64)          4672      
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 5, 5, 64)          36928     
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
dropout_73 (Dropout)         (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 2, 2, 128)         73856     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 1, 1, 128)       

In [183]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=5)
cnn.fit(x_train_cnn, y_train, batch_size=100, epochs=500, verbose=1, callbacks=[early_stopping], validation_data=(x_val_cnn, y_val))

Epoch 1/500
720/720 [==============================] - 3s 4ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0011 - val_mean_squared_error: 0.0011
Epoch 2/500
720/720 [==============================] - 2s 3ms/step - loss: 7.7658e-04 - mean_squared_error: 7.7658e-04 - val_loss: 6.4522e-04 - val_mean_squared_error: 6.4522e-04
Epoch 3/500
720/720 [==============================] - 2s 3ms/step - loss: 7.3012e-04 - mean_squared_error: 7.3012e-04 - val_loss: 5.8464e-04 - val_mean_squared_error: 5.8464e-04
Epoch 4/500
720/720 [==============================] - 2s 3ms/step - loss: 6.7848e-04 - mean_squared_error: 6.7848e-04 - val_loss: 8.6540e-04 - val_mean_squared_error: 8.6540e-04
Epoch 5/500
720/720 [==============================] - 3s 3ms/step - loss: 6.5146e-04 - mean_squared_error: 6.5146e-04 - val_loss: 5.2755e-04 - val_mean_squared_error: 5.2755e-04
Epoch 6/500
720/720 [==============================] - 2s 3ms/step - loss: 6.3325e-04 - mean_squared_error: 6.3325e-04 - 

In [354]:
y_val_pred_cnn = cnn.predict(x_val_cnn)
print(mean_squared_error(y_val, y_val_pred_cnn, squared=False))

0.02229915929187929


In [185]:
cnn.save(dir+'cnn2229.h5', include_optimizer=False)

## BiLSTM

In [268]:
bilstm = Sequential()
bilstm.add(Bidirectional(LSTM(
            units = 128, 
            activation = 'tanh', 
            return_sequences = True,
            input_shape = (x_train.shape[1], x_train.shape[2])
            )))
bilstm.add(Dropout(0.2))
bilstm.add(Bidirectional(LSTM(
            units = 128, 
            activation = 'tanh', 
            return_sequences = True,
            input_shape = (x_train.shape[1], x_train.shape[2])
            )))
bilstm.add(Dropout(0.2))
bilstm.add(Flatten())
bilstm.add(Dense(64))
bilstm.add(Dropout(0.2))
bilstm.add(Dense(64))
bilstm.add(Dense(1))
bilstm.compile(optimizer='adam', loss='mean_squared_error')
#bilstm.summary()

In [269]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3)
bilstm.fit(x_train, y_train, batch_size=100, epochs=500, verbose=1, callbacks=[early_stopping], validation_data=(x_val, y_val))

Epoch 1/500
720/720 [==============================] - 7s 10ms/step - loss: 0.0012 - val_loss: 8.2137e-04
Epoch 2/500
720/720 [==============================] - 6s 8ms/step - loss: 8.4056e-04 - val_loss: 7.8047e-04
Epoch 3/500
720/720 [==============================] - 6s 8ms/step - loss: 7.9305e-04 - val_loss: 7.2140e-04
Epoch 4/500
720/720 [==============================] - 6s 8ms/step - loss: 7.5774e-04 - val_loss: 6.8462e-04
Epoch 5/500
720/720 [==============================] - 6s 8ms/step - loss: 7.3475e-04 - val_loss: 6.7820e-04
Epoch 6/500
720/720 [==============================] - 6s 8ms/step - loss: 7.0332e-04 - val_loss: 6.4311e-04
Epoch 7/500
720/720 [==============================] - 6s 8ms/step - loss: 6.9492e-04 - val_loss: 6.7373e-04
Epoch 8/500
720/720 [==============================] - 6s 8ms/step - loss: 6.7833e-04 - val_loss: 6.1348e-04
Epoch 9/500
720/720 [==============================] - 6s 8ms/step - loss: 6.4551e-04 - val_loss: 5.8545e-04
Epoch 10/500
720/720 [

In [270]:
y_val_pred_bilstm = bilstm.predict(x_val)
print(mean_squared_error(y_val, y_val_pred_bilstm, squared=False))

0.024471656796671296


## BiLSTM Attention

In [304]:
x_train.shape

(72000, 8, 49)

In [396]:
bilstm = Sequential()
bilstm.add(Bidirectional(LSTM(
            units = 128, 
            activation = 'tanh', 
            return_sequences = True,
            input_shape = (x_train.shape[1], x_train.shape[2])
            )))
bilstm.add(SeqSelfAttention(
            attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
            attention_activation='sigmoid'))
bilstm.add(Dropout(0.2))
bilstm.add(Bidirectional(LSTM(
            units = 256, 
            activation = 'tanh', 
            return_sequences = True,
            input_shape = (x_train.shape[1], x_train.shape[2])
            )))
bilstm.add(SeqSelfAttention(
            attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
            attention_activation='sigmoid'))
bilstm.add(Flatten())
bilstm.add(Dense(256))
#bilstm.add(Dropout(0.5))
bilstm.add(Dense(128))
bilstm.add(Dense(1, activation = 'relu'))
bilstm.compile(optimizer='adam', loss='mean_squared_error')
bilstm.build((None, x_train.shape[1], x_train.shape[2]))
bilstm.summary()

Model: "sequential_104"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_89 (Bidirectio (None, 8, 256)            182272    
_________________________________________________________________
seq_self_attention_44 (SeqSe (None, 8, 256)            65537     
_________________________________________________________________
dropout_190 (Dropout)        (None, 8, 256)            0         
_________________________________________________________________
bidirectional_90 (Bidirectio (None, 8, 512)            1050624   
_________________________________________________________________
seq_self_attention_45 (SeqSe (None, 8, 512)            262145    
_________________________________________________________________
flatten_63 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_242 (Dense)            (None, 256)            

In [397]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3)
bilstm.fit(x_train, y_train, batch_size=100, epochs=500, verbose=1, callbacks=[early_stopping], validation_data=(x_val, y_val))

Epoch 1/500
720/720 [==============================] - 10s 14ms/step - loss: 0.0015 - val_loss: 9.8889e-04
Epoch 2/500
720/720 [==============================] - 9s 12ms/step - loss: 9.0982e-04 - val_loss: 8.8813e-04
Epoch 3/500
720/720 [==============================] - 9s 12ms/step - loss: 8.5636e-04 - val_loss: 7.7958e-04
Epoch 4/500
720/720 [==============================] - 9s 12ms/step - loss: 7.8345e-04 - val_loss: 0.0010
Epoch 5/500
720/720 [==============================] - 9s 12ms/step - loss: 7.0946e-04 - val_loss: 7.2461e-04
Epoch 6/500
720/720 [==============================] - 9s 12ms/step - loss: 6.6680e-04 - val_loss: 6.6113e-04
Epoch 7/500
720/720 [==============================] - 9s 12ms/step - loss: 6.6029e-04 - val_loss: 6.4701e-04
Epoch 8/500
720/720 [==============================] - 9s 12ms/step - loss: 6.2508e-04 - val_loss: 7.0055e-04
Epoch 9/500
720/720 [==============================] - 9s 12ms/step - loss: 5.9017e-04 - val_loss: 6.2975e-04
Epoch 10/500
720/

In [398]:
y_val_pred_bilstm = bilstm.predict(x_val)
print(mean_squared_error(y_val, y_val_pred_bilstm, squared=False))

0.022335474909951785


In [399]:
bilstm.save(dir+'bilstm2233.h5', include_optimizer=False)

## Ensemble

In [400]:
y_val_pred_mix = np.column_stack((y_val_pred_cnn, y_val_pred_bilstm)).mean(1)

In [413]:
print(mean_squared_error(y_val, y_val_pred_mix, squared=False))

0.02100536451200374


# Submission

In [353]:
current_best = keras.models.load_model(dir+'cnn2229.h5', compile=False)
y_val_pred_cnn = current_best.predict(x_val_cnn)
print(mean_squared_error(y_val, y_val_pred_cnn, squared=False))

0.02229915929187929


In [373]:
x_test_cnn = x_test.reshape(len(x_test), 8, 7, 7).transpose(0,2,3,1)
y_test_pred_cnn = current_best.predict(x_test_cnn)

In [402]:
y_test_pred_bilstm = bilstm.predict(x_test)

In [408]:
y_test_pred_mix = np.column_stack((y_test_pred_cnn, y_test_pred_bilstm)).mean(1)

In [414]:
pd.DataFrame(y_test_pred_mix).to_csv(dir+'MichiharuYamashita_labels.csv', header=False, index=False)

In [420]:
print(mean_squared_error(y_val_pred_mix, y_val_pred_cnn, squared=False))
print(mean_squared_error(y_val_pred_bilstm, y_val_pred_cnn, squared=False))
print(mean_squared_error(y_test_pred_bilstm, y_test_pred_cnn, squared=False))

0.007539074
0.015078147
0.014137645


In [415]:
y_test_pred_mix.reshape(1600, 1)

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.02536126],
       [0.02470684],
       [0.0207977 ]], dtype=float32)

In [411]:
y_test_pred_cnn

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.02164907],
       [0.02387166],
       [0.01954853]], dtype=float32)

In [412]:
y_test_pred_bilstm

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.02907345],
       [0.02554202],
       [0.02204687]], dtype=float32)